# Packages

In [1]:
from os import getcwd as gwd
from os import chdir as cd
from itertools import combinations 
from scipy.stats import pearsonr as pr
from scipy.stats import zscore as zscore
from scipy.stats import ttest_rel as ttest
import copy
import numpy as np
import pandas
import statsmodels.api as sm
import statsmodels.formula.api as smf
%load_ext rpy2.ipython
np.random.seed(223)

C:\Users\jerry\anaconda3\lib\site-packages\rpy2\robjects\pandas2ri.py:17: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


# Data

In [2]:
subjs = ['AN2','EW','IB','KH','MK','MY','NP']
dataAll = dict()
indcounter = 1
for subjName in subjs:
    data = pandas.read_csv((subjName+'.csv'),header=None,names=['cond','x','y','locs'],usecols=[13, 26, 27, 35])
    data = data[-pandas.isnull(data.x)] #delete the initial lines
    data = data[1:769] #take the rows we want
    data.index = range(768)
    data.cond[data.cond == 'womanmask.png'] = 1
    data.cond[data.cond == 'noisePatch.png'] = 2
    data = data.astype(float)
    condarray = np.empty(768, dtype = np.int)
    condarray.fill(indcounter)
    data['ind'] = condarray
    indcounter += 1
    dataAll[subjName] = data

In [3]:
dataAll['AN2']

,cond,x,y,locs,ind
0,1.0,-0.334549,-0.052747,73.0,1
1,1.0,0.335009,-0.048342,48.0,1
2,2.0,0.223459,-0.343859,40.0,1
3,2.0,-0.270573,-0.346350,31.0,1
4,1.0,0.272899,-0.136931,93.0,1
...,...,...,...,...,...
763,1.0,-0.290977,0.212096,67.0,1
764,2.0,0.075421,-0.296298,38.0,1
765,1.0,-0.014006,0.341997,60.0,1
766,2.0,-0.256411,0.230703,18.0,1


# Functions

In [4]:
#Calculate angular response
def calAng(x, y):
    angResp = np.rad2deg(np.arctan2(y,x))
    if angResp < 0:
        angResp += 360
    return angResp

#Morph errors---what exactly does this do???
def morphErr(err):
    if err > 180:
        err = -360+err
    else:
        if err < (-180):
            err = 360+err
    return err

#Replace outliers with mean
def clearOutliers(data):
    avg = np.mean(data, axis = None)
    sd = np.std(data, axis = None)
    out1 = np.nonzero(data < float(avg - 3*sd))
    out2 = np.nonzero(data > float(avg + 3*sd))
    if len(out1) != 0:
        data[out1] = avg
    if len(out2) != 0:
        data[out2] = avg
    return data


# Preprocessing

In [5]:
def prepersubj(data):
    angErr = None
    for trials in range(0, 768):
        err = morphErr(calAng(data.x[trials], data.y[trials]) - data.locs[trials]*7.5)
        if angErr is not None:
            angErr = np.vstack((angErr, np.array(err)))
        else:
            angErr = np.array(err)
    angErr = clearOutliers(angErr)
    return angErr

In [6]:
prepersubj(dataAll['EW'])

array([[-5.06061225e+00],
       [-3.95137043e-01],
       [ 1.01098118e-01],
       [-2.85366445e+00],
       [-1.22413000e+00],
       [ 5.19442886e+00],
       [-3.52624678e-01],
       [-8.93180295e-01],
       [ 3.18910473e+00],
       [-2.89187706e+00],
       [-2.73687993e+00],
       [-1.60193674e+00],
       [ 6.94172536e+00],
       [-9.62711057e-01],
       [ 1.67107585e+00],
       [ 7.84824643e-01],
       [-4.03945391e+00],
       [-3.95314635e+00],
       [-1.19509756e+00],
       [-7.03328523e+00],
       [ 4.73319416e+00],
       [-3.06101074e+00],
       [-2.93360010e+00],
       [ 1.08433760e+01],
       [-1.94276780e+00],
       [-2.51689348e+00],
       [ 6.27496994e+00],
       [ 4.33424518e-01],
       [ 6.68386411e+00],
       [ 4.77783137e+00],
       [ 4.89723159e+00],
       [-1.67353541e+00],
       [ 4.02434474e+00],
       [-2.88651619e-01],
       [-3.34889120e+00],
       [-4.69019191e+00],
       [-1.07299837e+00],
       [-4.59228870e+00],
       [-9.0

In [15]:
errAll = dict()
for subj in subjs:   
    errs = pandas.DataFrame(data = prepersubj(dataAll[subj]),columns=['errors'])
    errs['locs'] = dataAll[subj].locs
    errs['cond'] = dataAll[subj].cond
    errs['ind'] = dataAll[subj].ind
    errs = errs.sort_values('locs')
    errs.index = range(768)
    errAll[subj] = errs

In [16]:
errAll['AN2']

,errors,locs,cond,ind
0,-5.457612,0.0,2.0,1
1,-3.269334,0.0,2.0,1
2,-4.825476,0.0,2.0,1
3,-3.345750,0.0,2.0,1
4,-8.788197,0.0,2.0,1
...,...,...,...,...
763,-10.305166,95.0,1.0,1
764,-4.201660,95.0,1.0,1
765,-0.665093,95.0,1.0,1
766,-4.466523,95.0,1.0,1


In [17]:
table = pandas.DataFrame()
for subj in subjs:
    errAll[subj] = errAll[subj].groupby(by = 'locs', as_index = False).mean()
    table = pandas.concat([table, errAll[subj]], axis = 0)
table

,locs,errors,cond,ind
0,0.0,-5.431939,2.0,1
1,1.0,0.020900,2.0,1
2,2.0,0.966781,2.0,1
3,3.0,3.584510,2.0,1
4,4.0,4.989486,2.0,1
...,...,...,...,...
91,91.0,-1.739263,1.0,7
92,92.0,-3.640388,1.0,7
93,93.0,-3.084430,1.0,7
94,94.0,-16.697334,1.0,7


In [18]:
%%R -i table
library(nlme)
model <- lme(errors ~ locs + cond,random = ~1 | ind, data=table)
summary(model)


AttributeError: type object 'DataFrame' has no attribute 'from_items'